In [ ]:
import numpy as np
from collections import defaultdict
import copy

# モンテカルロ法
- 前回は動的計画法を用いて状態価値関数を実装したのち,方策反復法によって最適方策を得ることができた。
    - しかし動的計画法では計算量が膨大になってしまう恐れがある、また状態遷移確率と報酬が既知である必要がある。
        - 前回実装した環境では右に行こうとすれば必ず右に行くようになっていたが、そうはならない問題も存在する。
    - そのための代替案の一つがモンテカルロ法である。


In [ ]:
# 環境の実装
class GridWorld:
    def __init__(self):
        #行動は4種類(それぞれを数字に置き換える)
        self.action_list = [0,1,2,3]
        # 辞書型で行動を紐づける
        self.action_meaning = {
            0: "UP",
            1: "DOWN",
            2: "LEFT",
            3: "RIGHT"
        }
        # 以下にマップを作成する
        # それぞれの値はその場所の報酬を指す
        self.reward_map = np.array(
            [[0,0,0,1],
            [0,None,0,-1],
            [0,0,0,0]]
        )
        #以下に障害物を記載し、その座標を保持
        self.goal = (0,3)
        self.start = (2,0)
        self.wall = (1,1)
        #エージェントの場所の配列を保持(スタート値で初期化)
        self.agent_state = self.start

    # 以下にメソッドを実装
    # マップの情報を取得(主に再代入不可能のgetterの役割)
    # 高さを返却
    @property
    def height(self):
        return len(self.reward_map)
    # 横幅を返却
    @property
    def width(self):
        return len(self.reward_map[0])

    #行動の種類を返却
    @property
    def actions(self):
        return self.action_list

    def states(self):
        for h in range(self.height):
            for w in range(self.width):
                yield (h,w)


    # 以下にエージェントを動かすための関数と報酬関数を実装する。
    def next_step(self,state,action):
        # 上下左右の行動をそれぞれ座標に足し合わせることで実装
        action_list = [(-1,0),(1,0),(0,-1),(0,1)]
        # 0~3までの数字を配列に入れることで行動となる数値を取得
        move = action_list[action]
        # エージェントを移動させる(moveを現在の状態の座標に足し合わせる)
        # next_state = (y座標,x座標)となる
        next_state = (state[0] + move[0],state[1] + move[1])
        new_y,new_x = next_state

        # 移動したのちそれがマップから外れてるかどうかの確認処理(外れていたら更新を中断)
        if new_x < 0 or new_x >= self.width or new_y < 0 or new_y >= self.height or next_state == self.wall:
            next_state = state
        return next_state #(2,3)みたいに次の座標
    # 報酬関数
    def reward(self,state,action,next_state):
        return self.reward_map[next_state] # 1みたいに移動先での報酬を受け取る

    # 新しいコード
    # どう言った行動を起こしてどう言った報酬を受け取ったかを確認する。
    # これで行動を起こした後にどの場所に行ったかを確認できる
    def step(self, action):
        state = self.agent_state
        next_state = self.next_step(state, action)
        reward = self.reward(state, action, next_state)
        done = (next_state == self.goal)
